# Convolutional net finetuning

In [1]:
import numpy as np
import os

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from finetune import load_data, to_categorical

Using TensorFlow backend.


In [2]:
help(Dense)

Help on class Dense in module keras.layers.core:

class Dense(keras.engine.topology.Layer)
 |  Just your regular densely-connected NN layer.
 |  
 |  `Dense` implements the operation:
 |  `output = activation(dot(input, kernel) + bias)`
 |  where `activation` is the element-wise activation function
 |  passed as the `activation` argument, `kernel` is a weights matrix
 |  created by the layer, and `bias` is a bias vector created by the layer
 |  (only applicable if `use_bias` is `True`).
 |  
 |  Note: if the input to the layer has a rank greater than 2, then
 |  it is flattened prior to the initial dot product with `kernel`.
 |  
 |  # Example
 |  
 |  ```python
 |      # as first layer in a sequential model:
 |      model = Sequential()
 |      model.add(Dense(32, input_shape=(16,)))
 |      # now the model will take as input arrays of shape (*, 16)
 |      # and output arrays of shape (*, 32)
 |  
 |      # after the first layer, you don't need to specify
 |      # the size of the in

## Settings

In [3]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [4]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [10]:
# this is the model we will train
model = Model(base_model.input, predictions)
#model.summary()
model.layers.pop()

In [12]:
model.output_shape

(None, 2)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

## Get dataset

In [ ]:
meme_path = '/media/backup/crawling/memes/meme_characters/'
non_meme_path = '/media/backup/crawling/memes/test2014/'
m_train, m_test, n_train, n_test = load_data(meme_path, non_meme_path)

In [ ]:
# Training set
x_train = np.vstack([m_train, n_train])
my_train = np.ones((m_train.shape[0], 1))
ny_train = np.zeros((n_train.shape[0], 1))
y_train = to_categorical(np.vstack([my_train, ny_train]), num_classes=2)

In [ ]:
# Validation set
x_test = np.vstack([m_test, n_test])
my_test = np.ones((m_test.shape[0], 1))
ny_test = np.zeros((n_test.shape[0], 1))
y_test = to_categorical(np.vstack([my_test, ny_test]), num_classes=2)

## Testing while training

In [ ]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

## Callbacks

In [ ]:
logdir = 'inception_log2.0/'

In [ ]:
tb = TensorBoard(log_dir=logdir, histogram_freq=1, write_images=True)

## Training

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=32),
                    samples_per_epoch = len(x_train) / 32,
                    nb_epoch=80,
                    callbacks=[tb],
                    #initial_epoch=30,
                    validation_data=test_datagen.flow(x_test, y_test, batch_size=32),
                    nb_val_samples=1600 / 32)
model.save(os.path.join(logdir, 'fine_inception.h5'))

In [ ]:
score = model.evaluate_generator(test_datagen.flow(x_test, y_test, batch_size=32), 32)
print('score:', score)